In [2]:
N = ord("Y") % 5 + 1
N

5

In [7]:
import pandas as pd

file_path = 'lab6.xlsx' 
data = pd.read_excel(file_path)

data.head()

,N,file name,image size,glasses color,line width
0,1,emma-watson2.jpg,300x300,red,2
1,2,emma-watson.jpg,400x400,blue,3
2,3,draco.jpg,700x700,cian,4
3,4,longbottom.jpg,600x600,magenta,5
4,5,ron_wesley.jpg,500x500,yellow,6


In [10]:
import cv2
from PIL import Image, ImageDraw

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

def process_image(image_path, output_path, glasses_color, line_width):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=4)

    pil_img = Image.open(image_path)
    draw = ImageDraw.Draw(pil_img)

    for (x, y, w, h) in faces:
        face_roi = gray[y:y + h, x:x + w]

        eyes = eye_cascade.detectMultiScale(face_roi, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        if len(eyes) < 2:
            continue
        
        eyes = sorted(eyes, key=lambda ex: ex[0])[:2]
        if abs(eyes[0][1] - eyes[1][1]) > 20:
            continue

        left_eye = (x + eyes[0][0] + eyes[0][2] // 2, y + eyes[0][1] + eyes[0][3] // 2)
        right_eye = (x + eyes[1][0] + eyes[1][2] // 2, y + eyes[1][1] + eyes[1][3] // 2)

        radius = (right_eye[0] - left_eye[0]) // 3
        
        draw.ellipse((left_eye[0] - radius, left_eye[1] - radius, left_eye[0] + radius, left_eye[1] + radius),
                     outline=glasses_color, width=line_width)
        draw.ellipse((right_eye[0] - radius, right_eye[1] - radius, right_eye[0] + radius, right_eye[1] + radius),
                     outline=glasses_color, width=line_width)
        
        draw.line((left_eye[0] + radius, left_eye[1], right_eye[0] - radius, right_eye[1]), 
                  fill=glasses_color, width=line_width)

        draw.line((left_eye[0] - radius, left_eye[1], left_eye[0] - radius - 50, left_eye[1]), 
                  fill=glasses_color, width=line_width)
        draw.line((right_eye[0] + radius, right_eye[1], right_eye[0] + radius + 50, right_eye[1]), 
                  fill=glasses_color, width=line_width)

    pil_img.thumbnail((500, 500), Image.Resampling.LANCZOS)
    
    pil_img.save(output_path)

In [11]:
process_image("Images/ron_wesley.jpg", "ron_wesley_output.jpg", glasses_color="yellow", line_width=6)